<a href="https://www.kaggle.com/code/fareselmenshawii/lstm-from-scratch?scriptVersionId=121774265" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="padding:10px; 
            color:#FF9F00;
            margin:10px;
            font-size:150%;
            display:fill;
            border-radius:1px;
            border-style: solid;
            border-color:#FF9F00;
            background-color:#3E3D53;
            overflow:hidden;">
    <center>
        <a id='top'></a>
        <b>Table of Contents</b>
    </center>
    <br>
    <ul>
        <li>
            <a href="#1">1 -  Overview and Imports</a>
        </li>
        <li>
            <a href="#2">2 - Data Preparation</a>
        </li>
        <li>
            <a href="#3">3 - LSTM Implementation</a>
        <li>
            <a href="#4">4 - Thank you</a>
        </li> 
        <li>
            <a href="#5">5 - References</a>
        </li>
    </ul>
</div>
<a id="1"></a>

<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Overview and Imports</center></h1>
    
# Overview and Imports

**Long Short-Term Memory (LSTM) networks are a type of Recurrent Neural Network (RNN) that are designed to handle sequential data, such as time series or natural language text. They achieve this by using a memory cell that can selectively remember or forget information at each time step of the input sequence, allowing the network to maintain a memory of previous inputs over a longer period of time.**

**This notebook contains an implementation of an LSTM that can be used for language modeling. The self takes in a sequence of characters and outputs the probability distribution over the next character in the sequence. The network is trained on a corpus of text and then used to generate new text that has a similar distribution of characters as the training corpus.**

In [1]:
import os
import numpy as np
import scipy as sp

 <a id="2"></a>
<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Data Preparation</center></h1>
    
# Data Prepartion

In [2]:
class DataGenerator:
    """
    A class for reading and preprocessing text data.
    """

    def __init__(self, path: str, sequence_length: int):
        """
        Initializes a DataReader object with the path to a text file and the desired sequence length.

        Args:
            path (str): The path to the text file.
            sequence_length (int): The length of the sequences that will be fed to the self.
        """
        with open(path) as f:
            # Read the contents of the file
            self.data = f.read()

        # Find all unique characters in the text
        chars = list(set(self.data))

        # Create dictionaries to map characters to indices and vice versa
        self.char_to_idx = {ch: i for (i, ch) in enumerate(chars)}
        self.idx_to_char = {i: ch for (i, ch) in enumerate(chars)}

        # Store the size of the text data and the size of the vocabulary
        self.data_size = len(self.data)
        self.vocab_size = len(chars)

        # Initialize the pointer that will be used to generate sequences
        self.pointer = 0

        # Store the desired sequence length
        self.sequence_length = sequence_length


    def next_batch(self):
        """
        Generates a batch of input and target sequences.

        Returns:
            inputs_one_hot (np.ndarray): A numpy array with shape `(batch_size, vocab_size)` where each row is a one-hot encoded representation of a character in the input sequence.
            targets (list): A list of integers that correspond to the indices of the characters in the target sequence, which is the same as the input sequence shifted by one position to the right.
        """
        input_start = self.pointer
        input_end = self.pointer + self.sequence_length

        # Get the input sequence as a list of integers
        inputs = [self.char_to_idx[ch] for ch in self.data[input_start:input_end]]

        # One-hot encode the input sequence
        inputs_one_hot = np.zeros((len(inputs), self.vocab_size))
        inputs_one_hot[np.arange(len(inputs)), inputs] = 1

        # Get the target sequence as a list of integers
        targets = [self.char_to_idx[ch] for ch in self.data[input_start + 1:input_end + 1]]

        # Update the pointer
        self.pointer += self.sequence_length

        # Reset the pointer if the next batch would exceed the length of the text data
        if self.pointer + self.sequence_length + 1 >= self.data_size:
            self.pointer = 0

        return inputs_one_hot, targets

<a id="3"></a>
<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>LSTM Implementation</center></h1>


# LSTM Implementation


## Forward Propagation

**The forward pass is a step in training a simple LSTM (Long Short-Term Memory) model. During the forward pass, the model takes an input sequence X and performs a series of computations to generate a sequence of output probability vectors y_pred, which represent the predicted probability distribution over the possible output classes for each time step in the input sequence.**

**The LSTM model has a memory cell that stores information from previous time steps, which is updated based on the input sequence and a set of learned parameters. During the forward pass, the LSTM computes a set of vectors representing the forget gate, input gate, candidate cell state, and output gate for each time step in the input sequence.**

**These vectors are used to update the cell state and hidden state for each time step, which are then used to generate the output probability vectors. The function returns the input sequence, the cell state for each time step, the forget gate, input gate, output gate, candidate cell state, hidden state, and output probability vector for each time step.**

**Here are the formulas used in the code:**

**Concatenation of the input and hidden state:**
$$\text{concat} = \begin{bmatrix} a_{t-1} \ x_t \end{bmatrix}$$

**Forget gate:**
**$$f_t = \sigma(W_f \text{concat} + b_f)$$**

**Input gate:**
**$$i_t = \sigma(W_i \text{concat} + b_i)$$**

**Candidate cell state:**
**$$\tilde{c}_t = \tanh(W_c \text{concat} + b_c)$$**

**Cell state:**
**$$c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$$**

**Output gate:**
**$$o_t = \sigma(W_o \text{concat} + b_o)$$**

**Hidden state:**
**$$a_t = o_t \odot \tanh(c_t)$$**

**Output probability vector:**
**$$\hat{y}_t = \text{softmax}(W_y a_t + b_y)$$**

**Where:**

**$x_t$ is the input vector at time $t$.**

**$a_{t-1}$ is the hidden state vector at time $t-1$.**

**$f_t$, $i_t$, $o_t$ are the forget, input, and output gates at time $t$, respectively.**

**$c_t$ is the cell state vector at time $t$.**

**$\tilde{c}_t$ is the candidate cell state at time $t$.**

$\hat{y}_t$ is the output probability vector at time $t$.

**$\odot$ is the element-wise multiplication operator.**

**$\sigma$ is the sigmoid activation function.**

**$\text{tanh}$ is the hyperbolic tangent activation function.**

**$W_f, W_i, W_c, W_o, W_y$ are the weight matrices for the forget, input, candidate, output, and output layers, respectively.**

**$b_f, b_i, b_c, b_o, b_y$ are the bias vectors for the forget, input, candidate, output, and output layers, respectively.**
****
## Backpropagation
**The backpropagation step in training a simple LSTM model involves computing the gradients of the loss with respect to the model parameters, which are used to update the parameters in the opposite direction of the gradient to minimize the loss function.**

**In this particular implementation of the LSTM model, the backpropagation step is implemented using the standard backpropagation through time (BPTT) algorithm. The algorithm iteratively computes the gradients of the loss with respect to each parameter in the model by propagating the gradients backwards through time from the output sequence to the input sequence.**

**Starting from the final time step, the gradients of the loss with respect to the output probability vectors are computed using the cross-entropy loss function. The gradients are then propagated backwards through time by computing the gradients of the loss with respect to the hidden state and cell state for each time step.**

**These gradients are used to compute the gradients of the loss with respect to the output gate, candidate cell state, input gate, and forget gate vectors for each time step, which are in turn used to compute the gradients of the loss with respect to the weight matrices and bias vectors for each gate and the output layer.**

**The gradients are then accumulated across all time steps and used to update the model parameters using an optimization algorithm, such as stochastic gradient descent (SGD). This process is repeated for each batch of input sequences during training, until the model converges to a set of optimal parameters that minimize the loss function.**

$$\frac{\partial \mathcal{L}}{\partial \mathbf{W_y}} = \sum_{t=1}^T \mathbf{a}_t \cdot (\mathbf{y}_t - \mathbf{t}_t)^\top$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{b_y}} = \sum_{t=1}^T (\mathbf{y}_t - \mathbf{t}_t)$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{a}_t} = \mathbf{W_y}^\top \cdot (\mathbf{y}_t - \mathbf{t}t) + \frac{\partial \mathcal{L}}{\partial \mathbf{a}{t+1}} \cdot \mathbf{W_f}^\top \cdot \mathbf{f}_t + \frac{\partial \mathcal{L}}{\partial \mathbf{c}_t} \cdot \mathbf{W_c}^\top \cdot \mathbf{i}_t + \frac{\partial \mathcal{L}}{\partial \mathbf{o}_t} \cdot \mathbf{W_o}^\top \cdot \mathbf{o}_t$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{c}t} = \frac{\partial \mathcal{L}}{\partial \mathbf{c}{t+1}} \cdot \mathbf{f}_t + \mathbf{i}_t \cdot \mathbf{c}_t \cdot (1 - \mathbf{c}_t) \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{a}_t}$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{i}_t} = \mathbf{c}_t \cdot (1 - \mathbf{i}_t) \cdot \mathbf{i}_t \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{c}_t}$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{f}t} = \mathbf{c}{t-1} \cdot (1 - \mathbf{f}_t) \cdot \mathbf{f}_t \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{c}_t}$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{o}_t} = \mathbf{a}_t \cdot (1 - \mathbf{o}_t) \cdot \mathbf{o}_t \cdot \frac{\partial \mathcal{L}}{\partial \mathbf{c}_t}$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{c}_{t-1}} = \frac{\partial \mathcal{L}}{\partial \mathbf{c}_t} \cdot \mathbf{f}_t$$

$$\frac{\partial \mathcal{L}}{\partial \mathbf{W_c}} = \sum_{t=1}^T \frac{\partial \mathcal{L}}{\partial \mathbf{c}_t} \cdot \mathbf{i}_t \cdot (1 - \mathbf{c}_t^2) \cdot \mathbf{concat}_t^\top$$
****
## Loss:

**The cross-entropy loss between the predicted probabilities y_pred and the true targets y_true at a single time step $t$ is:**

**$$H(y_{true,t}, y_{pred,t}) = -\sum_i y_{true,t,i} \log(y_{pred,t,i})$$**

**where $y_{pred,t}$ is the predicted probability distribution at time step $t$, $y_{true,t}$ is the true probability distribution at time step $t$ (i.e., a one-hot encoded vector representing the true target), and $i$ ranges over the vocabulary size.**

**The total loss is then computed as the sum of the cross-entropy losses over all time steps:**

**$$L = \sum_{t=1}^{T} H(y_{true,t}, y_{pred,t})$$**

**where $T$ is the sequence length.**
 
****
## Optimization


**The AdamW optimizer is an extension of the Adam optimizer that incorporates weight decay directly into the update rule, rather than treating it as a separate regularization term. This has been shown to improve the generalization performance of deep neural networks.**

**To use the AdamW optimizer in practice, we first initialize the parameters $W_f$, $b_f$, $W_i$, $b_i$, $W_c$, $b_c$, $W_o$, and $b_o$ with small random values. Then, for each training iteration, we compute the gradients of the loss function with respect to the parameters using backpropagation. These gradients are then used to update the parameters using the AdamW update rules described above. This process is repeated for a fixed number of iterations or until convergence is achieved.**

**In summary, the AdamW optimizer is a powerful optimization algorithm for training deep neural networks. By incorporating weight decay directly into the update rule, it can improve the generalization performance of the network and reduce the risk of overfitting.**

**For the AdamW update of parameter Wf:**

$m_{Wf} \leftarrow \beta_1 m_{Wf} + (1 - \beta_1) \nabla_{Wf} J$

$v_{Wf} \leftarrow \beta_2 v_{Wf} + (1 - \beta_2) (\nabla_{Wf} J)^2$

$\hat{m}{Wf} = \frac{m{Wf}}{1 - \beta_1}$

$\hat{v}{Wf} = \frac{v{Wf}}{1 - \beta_2}$

$W_f \leftarrow W_f - \eta \frac{\hat{m}{Wf}}{\sqrt{\hat{v}{Wf}} + \epsilon} - \eta \lambda_2 W_f$

**For the AdamW update of parameter bf:**

$m_{bf} \leftarrow \beta_1 m_{bf} + (1 - \beta_1) \nabla_{bf} J$

$v_{bf} \leftarrow \beta_2 v_{bf} + (1 - \beta_2) (\nabla_{bf} J)^2$

$\hat{m}{bf} = \frac{m{bf}}{1 - \beta_1}$

$\hat{v}{bf} = \frac{v{bf}}{1 - \beta_2}$

$b_f \leftarrow b_f - \eta \frac{\hat{m}{bf}}{\sqrt{\hat{v}{bf}} + \epsilon} - \eta \lambda_2 b_f$

**For the AdamW update of parameter Wi:**

$m_{Wi} \leftarrow \beta_1 m_{Wi} + (1 - \beta_1) \nabla_{Wi} J$

$v_{Wi} \leftarrow \beta_2 v_{Wi} + (1 - \beta_2) (\nabla_{Wi} J)^2$

$\hat{m}{Wi} = \frac{m{Wi}}{1 - \beta_1}$

$\hat{v}{Wi} = \frac{v{Wi}}{1 - \beta_2}$

$W_i \leftarrow W_i - \eta \frac{\hat{m}{Wi}}{\sqrt{\hat{v}{Wi}} + \epsilon} - \eta \lambda_2 W_i$

**For the AdamW update of parameter bi:**

$m_{bi} \leftarrow \beta_1 m_{bi} + (1 - \beta_1) \nabla_{bi} J$

$v_{bi} \leftarrow \beta_2 v_{bi} + (1 - \beta_2) (\nabla_{bi} J)^2$

$\hat{m}{bi} = \frac{m{bi}}{1 - \beta_1}$

$\hat{v}{bi} = \frac{v{bi}}{1 - \beta_2}$

$b_i \leftarrow b_i - \eta \frac{\hat{m}{bi}}{\sqrt{\hat{v}{bi}} + \epsilon} - \eta \lambda_2 b_i$

**For the AdamW update of parameter Wc:**

$m_{Wc} \leftarrow \beta_1 m_{Wc} + (1 - \beta_1) \nabla_{Wc} J$

$v_{Wc} \leftarrow \beta_2 v_{Wc} + (1 - \beta2) (\nabla{Wc} J)^2$

$\hat{m}{Wc} = \frac{m{Wc}}{1 - \beta_1}$

$\hat{v}{Wc} = \frac{v{Wc}}{1 - \beta_2}$

$W_c \leftarrow W_c - \eta \frac{\hat{m}{Wc}}{\sqrt{\hat{v}{Wc}} + \epsilon} - \eta \lambda_2 W_c$

**For the AdamW update of parameter bc:**

$m_{bc} \leftarrow \beta_1 m_{bc} + (1 - \beta_1) \nabla_{bc} J$

$v_{bc} \leftarrow \beta_2 v_{bc} + (1 - \beta_2) (\nabla_{bc} J)^2$

$\hat{m}{bc} = \frac{m{bc}}{1 - \beta_1}$

$\hat{v}{bc} = \frac{v{bc}}{1 - \beta_2}$

$b_c \leftarrow b_c - \eta \frac{\hat{m}{bc}}{\sqrt{\hat{v}{bc}} + \epsilon} - \eta \lambda_2 b_c$

**For the AdamW update of parameter Wo:**

$m_{Wo} \leftarrow \beta_1 m_{Wo} + (1 - \beta_1) \nabla_{Wo} J$

$v_{Wo} \leftarrow \beta_2 v_{Wo} + (1 - \beta_2) (\nabla_{Wo} J)^2$

$\hat{m}{Wo} = \frac{m{Wo}}{1 - \beta_1}$

$\hat{v}{Wo} = \frac{v{Wo}}{1 - \beta_2}$

$W_o \leftarrow W_o - \eta \frac{\hat{m}{Wo}}{\sqrt{\hat{v}{Wo}} + \epsilon} - \eta \lambda_2 W_o$

**For the AdamW update of parameter bo:**

$m_{bo} \leftarrow \beta_1 m_{bo} + (1 - \beta_1) \nabla_{bo} J$

$v_{bo} \leftarrow \beta_2 v_{bo} + (1 - \beta_2) (\nabla_{bo} J)^2$

$\hat{m}{bo} = \frac{m{bo}}{1 - \beta_1}$

$\hat{v}{bo} = \frac{v{bo}}{1 - \beta_2}$

**$b_o \leftarrow b_o - \eta \frac{\hat{m}{bo}}{\sqrt{\hat{v}{bo}} + \epsilon} - \eta \lambda_2 b_o$**

**In the above formulas, $\nabla_{Wf} J$, $\nabla_{bf} J$, $\nabla_{Wi} J$, $\nabla_{bi} J$, $\nabla_{Wc} J$, $\nabla_{bc} J$, $\nabla_{Wo} J$, and $\nabla_{bo} J$ are the gradients of the loss function J with respect to the parameters $W_f$, $b_f$, $W_i$, $b_i$, $W_c$, $b_c$, $W_o$, and $b_o$ respectively. $\beta_1$ and $\beta_2$ are hyperparameters controlling the exponential decay rates of the moving averages. $\eta$ is the learning rate, $\lambda_2$ is the weight decay hyperparameter, and $\epsilon$ is a small constant to avoid division by zero.**



****
## Train

**The train method trains the LSTM on a dataset using backpropagation through time. The method takes an instance of DataReader containing the training data as input. The method initializes a hidden state vector a_prev at the beginning of each sequence to zero.**

**It then iterates until the smooth loss is less than a threshold value. During each iteration, it retrieves a batch of inputs and targets from the data generator.**

**The LSTM then performs a forward pass on the input sequence and computes the output probabilities. The backward pass is performed using the targets and output probabilities to calculate the gradients of the parameters of the network.**

**The AdamW algorithm is used to update the weights of the network. The method then calculates and updates the loss using the updated weights. The previous hidden state is updated for the next batch.**

**The method prints the progress every 10000 iterations by generating a sample of text using the sample method and printing the loss.**

**The train method can be summarized by the following steps:**

**$1.$ Initialize $a_{prev}$ to zero at the beginning of each sequence.**

**$2.$ Retrieve a batch of inputs and targets from the data generator.**

**$3.$ Perform a forward pass on the input sequence and compute the output probabilities.**

**$4.$ Perform a backward pass using the targets and output probabilities to calculate the gradients of the parameters of the network.**

**$5.$ Use the AdamW algorithm to update the weights of the network.**

**$6.$ Calculate and update the loss using the updated weights.**

**$7.$ Update the previous hidden state for the next batch.**

**$8.$ Print progress every 1000 iterations by generating a sample of text using the sample method and printing the loss.**

**$9.$ Repeat steps $2$-$8$ until the smooth loss is less than the threshold value.**

In [3]:
 class LSTM:
    """
    A class used to represent a Recurrent Neural Network (LSTM).

    Attributes
    ----------
    hidden_size : int
        The number of hidden units in the LSTM.
    vocab_size : int
        The size of the vocabulary used by the LSTM.
    sequence_length : int
        The length of the input sequences fed to the LSTM.
    self.learning_rate : float
        The learning rate used during training.

    Methods
    -------
    __init__(hidden_size, vocab_size, sequence_length, self.learning_rate)
        Initializes an instance of the LSTM class.
    """

    def __init__(self, hidden_size, vocab_size, sequence_length, learning_rate):
        """
        Initializes an instance of the LSTM class.

        Parameters
        ----------
        hidden_size : int
            The number of hidden units in the LSTM.
        vocab_size : int
            The size of the vocabulary used by the LSTM.
        sequence_length : int
            The length of the input sequences fed to the LSTM.
        learning_rate : float
            The learning rate used during training.
        """
        # hyper parameters
        self.mby = None
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate
        
        # model parameters
        self.Wf = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.bf = np.zeros((hidden_size, 1))
        
        self.Wi = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.bi = np.zeros((hidden_size, 1))

        self.Wc = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.bc = np.zeros((hidden_size, 1))
            
        self.Wo = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.bo = np.zeros((hidden_size, 1))
        
        self.Wy = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (vocab_size, hidden_size))
        self.by = np.zeros((vocab_size, 1))

        # initialize parameters for adamw optimizer
        self.mWf = np.zeros_like(self.Wf)
        self.vWf = np.zeros_like(self.Wf)
        self.mWi = np.zeros_like(self.Wi)
        self.vWi = np.zeros_like(self.Wi)
        self.mWc = np.zeros_like(self.Wc)
        self.vWc = np.zeros_like(self.Wc)
        self.mWo = np.zeros_like(self.Wo)
        self.vWo = np.zeros_like(self.Wo)
        self.mWy = np.zeros_like(self.Wy)
        self.vWy = np.zeros_like(self.Wy)
        self.mbf = np.zeros_like(self.bf)
        self.vbf = np.zeros_like(self.bf)
        self.mbi = np.zeros_like(self.bi)
        self.vbi = np.zeros_like(self.bi)
        self.mbc = np.zeros_like(self.bc)
        self.vbc = np.zeros_like(self.bc)
        self.mbo = np.zeros_like(self.bo)
        self.vbo = np.zeros_like(self.bo)
        self.mby = np.zeros_like(self.by)
        self.vby = np.zeros_like(self.by)

    def sigmoid(self, x):
        """
        Computes the sigmoid activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the sigmoid activation values.
        """
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        """
        Computes the softmax activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the softmax activation values.
        """
        # shift the input to prevent overflow when computing the exponentials
        x = x - np.max(x)
        # compute the exponentials of the shifted input
        p = np.exp(x)
        # normalize the exponentials by dividing by their sum
        return p / np.sum(p)

    def loss(self, y_preds, targets):
        """
        Computes the cross-entropy loss for a given sequence of predicted probabilities and true targets.

        Parameters:
            y_preds (ndarray): Array of shape (sequence_length, vocab_size) containing the predicted probabilities for each time step.
            targets (ndarray): Array of shape (sequence_length, 1) containing the true targets for each time step.

        Returns:
            float: Cross-entropy loss.
        """
        # calculate cross-entropy loss
        return sum(-np.log(y_preds[t][targets[t], 0]) for t in range(self.sequence_length))
    
    
    def adamw(self, beta1=0.9, beta2=0.999, epsilon=1e-8, L2_reg=1e-4):
        """
        Updates the LSTM's parameters using the AdamW optimization algorithm.
        """
        # AdamW update for Wf
        self.mWf = beta1 * self.mWf + (1 - beta1) * self.dWf
        self.vWf = beta2 * self.vWf + (1 - beta2) * np.square(self.dWf)
        m_hat = self.mWf / (1 - beta1)
        v_hat = self.vWf / (1 - beta2)
        self.Wf -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wf)

        # AdamW update for bf
        self.mbf = beta1 * self.mbf + (1 - beta1) * self.dbf
        self.vbf = beta2 * self.vbf + (1 - beta2) * np.square(self.dbf)
        m_hat = self.mbf / (1 - beta1)
        v_hat = self.vbf / (1 - beta2)
        self.bf -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.bf)

        # AdamW update for Wi
        self.mWi = beta1 * self.mWi + (1 - beta1) * self.dWi
        self.vWi = beta2 * self.vWi + (1 - beta2) * np.square(self.dWi)
        m_hat = self.mWi / (1 - beta1)
        v_hat = self.vWi / (1 - beta2)
        self.Wi -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wi)

        # AdamW update for bi
        self.mbi = beta1 * self.mbi + (1 - beta1) * self.dbi
        self.vbi = beta2 * self.vbi + (1 - beta2) * np.square(self.dbi)
        m_hat = self.mbi / (1 - beta1)
        v_hat = self.vbi / (1 - beta2)
        self.bi -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.bi)

        # AdamW update for Wc
        self.mWc = beta1 * self.mWc + (1 - beta1) * self.dWc
        self.vWc = beta2 * self.vWc + (1 - beta2) * np.square(self.dWc)
        m_hat = self.mWc / (1 - beta1)
        v_hat = self.vWc / (1 - beta2)
        self.Wc -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wc)

        # AdamW update for bc
        self.mbc = beta1 * self.mbc + (1 - beta1) * self.dbc
        self.vbc = beta2 * self.vbc + (1 - beta2) * np.square(self.dbc)
        m_hat = self.mbc / (1 - beta1)
        v_hat = self.vbc / (1 - beta2)
        self.bc -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.bc)

        # AdamW update for Wy
        self.mWy = beta1 * self.mWy + (1 - beta1) * self.dWy
        self.vWy = beta2 * self.vWy + (1 - beta2) * np.square(self.dWy)
        m_hat = self.mWy / (1 - beta1)
        v_hat = self.vWy / (1 - beta2)
        self.Wy -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wy)
        # AdamW update for by
        self.mby = beta1 * self.mby + (1 - beta1) * self.dby
        self.vby = beta2 * self.vby + (1 - beta2) * np.square(self.dby)
        m_hat = self.mby / (1 - beta1)
        v_hat = self.vby / (1 - beta2)
        self.by -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.by)


    def forward(self, X, c_prev, a_prev):
        """
        Performs forward propagation for a simple LSTM model.

        Args:
            X (numpy array): Input sequence, shape (sequence_length, input_size)
            c_prev (numpy array): Previous cell state, shape (hidden_size, 1)
            a_prev (numpy array): Previous hidden state, shape (hidden_size, 1)

        Returns:
            X (numpy array): Input sequence, shape (sequence_length, input_size)
            c (dictionary): Cell state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            f (dictionary): Forget gate for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            i (dictionary): Input gate for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            o (dictionary): Output gate for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            cc (dictionary): Candidate cell state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            a (dictionary): Hidden state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            y_pred (dictionary): Output probability vector for each time step, keys = time step, values = numpy array shape (output_size, 1)
        """
        # initialize dictionaries for backpropagation 
        c, f, i, o, cc, a, y_pred = {}, {}, {}, {}, {}, {}, {}
        c[-1] = np.copy(c_prev)  # store the initial cell state in the dictionary
        a[-1] = np.copy(a_prev)  # store the initial hidden state in the dictionary

        # iterate over each time step in the input sequence
        for t in range(X.shape[0]):
            # concatenate the input and hidden state
            xt = X[t, :].reshape(-1, 1)
            concat = np.vstack((a[t - 1], xt))

            # compute the forget gate
            f[t] = self.sigmoid(np.dot(self.Wf, concat) + self.bf)

            # compute the input gate
            i[t] = self.sigmoid(np.dot(self.Wi, concat) + self.bi)

            # compute the candidate cell state
            cc[t] = np.tanh(np.dot(self.Wc, concat) + self.bc)

            # compute the cell state
            c[t] = f[t] * c[t - 1] + i[t] * cc[t]

            # compute the output gate
            o[t] = self.sigmoid(np.dot(self.Wo, concat) + self.bo)

            # compute the hidden state
            a[t] = o[t] * np.tanh(c[t])

            # compute the output probability vector
            y_pred[t] = self.softmax(np.dot(self.Wy, a[t]) + self.by)

        # return the output probability vectors, cell state, hidden state and gate vectors
        return X, y_pred, c, f, i, o, cc, a 
    
    
    def backward(self, X, targets, y_pred, c_prev, a_prev, c, f, i, o, cc, a):
        """
        Performs backward propagation through time for an LSTM network.

        Args:
        - X: input data for each time step, with shape (sequence_length, input_size)
        - targets: target outputs for each time step, with shape (sequence_length, output_size)
        - y_pred: predicted outputs for each time step, with shape (sequence_length, output_size)
        - c_prev: previous cell state, with shape (hidden_size, 1)
        - a_prev: previous hidden state, with shape (hidden_size, 1)
        - c: cell state for each time step, with shape (sequence_length, hidden_size)
        - f: forget gate output for each time step, with shape (sequence_length, hidden_size)
        - i: input gate output for each time step, with shape (sequence_length, hidden_size)
        - o: output gate output for each time step, with shape (sequence_length, hidden_size)
        - cc: candidate cell state for each time step, with shape (sequence_length, hidden_size)
        - a: hidden state output for each time step, with shape (sequence_length, hidden_size)
        Returns:
            None
        """
        
        # initialize gradients for each parameter
        self.dWf, self.dWi, self.dWc, self.dWo, self.dWy = np.zeros_like(self.Wf), np.zeros_like(self.Wi), np.zeros_like(self.Wc), np.zeros_like(self.Wo), np.zeros_like(self.Wy)
        self.dbf, self.dbi, self.dbc, self.dbo, self.dby = np.zeros_like(self.bf), np.zeros_like(self.bi), np.zeros_like(self.bc), np.zeros_like(self.bo), np.zeros_like(self.by)
        dc_next = np.zeros_like(c_prev)
        da_next = np.zeros_like(a_prev)

        # iterate backwards through time steps
        for t in reversed(range(X.shape[0])):
            # compute the gradient of the output probability vector
            dy = np.copy(y_pred[t])
            dy[targets[t]] -= 1

            # compute the gradient of the output layer weights and biases
            self.dWy += np.dot(dy, a[t].T)
            self.dby += dy

            # compute the gradient of the hidden state
            da = np.dot(self.Wy.T, dy) + da_next
            dc = dc_next + (1 - np.tanh(c[t])**2) * o[t] * da
            
            # compute the gradient of the output gate
            xt = X[t, :].reshape(-1, 1)
            concat = np.vstack((a[t - 1], xt))
            do = o[t] * (1 - o[t]) * np.tanh(c[t]) * da
            self.dWo += np.dot(do, concat.T)
            self.dbo += do

            # compute the gradient of the candidate cell state
            dcc = dc * i[t] * (1 - np.tanh(cc[t])**2)
            self.dWc += np.dot(dcc, concat.T)
            self.dbc += dcc

            # compute the gradient of the input gate
            di = i[t] * (1 - i[t]) * cc[t] * dc
            self.dWi += np.dot(di, concat.T)
            self.dbi += di

            # compute the gradient of the forget gate
            df = f[t] * (1 - f[t]) * c[t - 1] * dc
            self.dWf += np.dot(df, concat.T)
            self.dbf += df

            # compute the gradient of the input to the current hidden state and cell state
            da_next = np.dot(self.Wf[:, :self.hidden_size].T, df)\
            + np.dot(self.Wi[:, :self.hidden_size].T, di)\
            + np.dot(self.Wc[:, :self.hidden_size].T, dcc)\
            + np.dot(self.Wo[:, :self.hidden_size].T, do)
            dc_next = dc * f[t]

        # clip gradients to avoid exploding gradients
        for grad in [self.dWf, self.dWi, self.dWc, self.dWo, self.dWy, self.dbf, self.dbi, self.dbc, self.dbo, self.dby]:
            np.clip(grad, -1, 1, out=grad)


    def train(self, data_generator):
        """
        Train the LSTM on a dataset using backpropagation through time.

        Args:
            data_generator: An instance of DataGenerator containing the training data.

        Returns:
            None
        """
        iter_num = 0
        # stopping criterion for training
        threshold = 46
        smooth_loss = -np.log(1.0 / data_generator.vocab_size) * self.sequence_length  # initialize loss
        while (smooth_loss > threshold):
            # initialize hidden state at the beginning of each sequence
            if data_generator.pointer == 0:
                c_prev = np.zeros((self.hidden_size, 1))
                a_prev = np.zeros((self.hidden_size, 1))

            # get a batch of inputs and targets
            inputs, targets = data_generator.next_batch()

            # forward pass
            X, y_pred, c, f, i, o, cc, a   = self.forward(inputs, c_prev, a_prev)
        
            # backward pass
            self.backward( X, targets, y_pred, c_prev, a_prev, c, f, i, o, cc, a)

            # calculate and update loss
            loss = self.loss(y_pred, targets)
            self.adamw()
            smooth_loss = smooth_loss * 0.999 + loss * 0.001
            # update previous hidden state for the next batch
            a_prev = a[self.sequence_length - 1]
            c_prev = c[self.sequence_length - 1]
            # print progress every 1000 iterations
            if iter_num % 1000 == 0:
                self.learning_rate *= 0.99
                sample_idx = self.sample(c_prev, a_prev, inputs[0, :], 200)
                print(''.join(data_generator.idx_to_char[idx] for idx in sample_idx))
                print("\n\niter :%d, loss:%f" % (iter_num, smooth_loss))
            iter_num += 1
    
            
    def sample(self, c_prev, a_prev, seed_idx, n):
        """
        Sample a sequence of integers from the model.

        Args:
            c_prev (numpy.ndarray): Previous cell state, a numpy array of shape (hidden_size, 1).
            a_prev (numpy.ndarray): Previous hidden state, a numpy array of shape (hidden_size, 1).
            seed_idx (numpy.ndarray): Seed letter from the first time step, a numpy array of shape (vocab_size, 1).
            n (int): Number of characters to generate.

        Returns:
            list: A list of integers representing the generated sequence.

        """
        # initialize input and seed_idx
        x = np.zeros((self.vocab_size, 1))
        # convert one-hot encoding to integer index
        seed_idx = np.argmax(seed_idx, axis=-1)

        # set the seed letter as the input for the first time step
        x[seed_idx] = 1

        # generate sequence of characters
        idxes = []
        c = np.copy(c_prev)
        a = np.copy(a_prev)
        for t in range(n):
            # compute the hidden state and cell state
            concat = np.vstack((a, x))
            i = self.sigmoid(np.dot(self.Wi, concat) + self.bi)
            f = self.sigmoid(np.dot(self.Wf, concat) + self.bf)
            cc = np.tanh(np.dot(self.Wc, concat) + self.bc)
            c = f * c + i * cc
            o = self.sigmoid(np.dot(self.Wo, concat) + self.bo)
            a = o * np.tanh(c)

            # compute the output probabilities
            y = self.softmax(np.dot(self.Wy, a) + self.by)

            # sample the next character from the output probabilities
            idx = np.random.choice(range(self.vocab_size), p=y.ravel())

            # set the input for the next time step
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1

            # append the sampled character to the sequence
            idxes.append(idx)

        # return the generated sequence
        return idxes


    def predict(self, data_generator, start, n):
        """
        Generate a sequence of n characters using the trained LSTM model, starting from the given start sequence.

        Args:
        - data_generator: an instance of DataGenerator
        - start: a string containing the start sequence
        - n: an integer indicating the length of the generated sequence

        Returns:
        - txt: a string containing the generated sequence
        """
        # initialize input sequence
        x = np.zeros((self.vocab_size, 1))
        chars = [ch for ch in start]
        idxes = []
        for i in range(len(chars)):
            idx = data_generator.char_to_idx[chars[i]]
            x[idx] = 1
            idxes.append(idx)
        # initialize cell state and hidden state
        a = np.zeros((self.hidden_size, 1))
        c = np.zeros((self.hidden_size, 1))
            
        # generate new sequence of characters
        for t in range(n):
            # compute the hidden state and cell state
            concat = np.vstack((a, x))
            i = self.sigmoid(np.dot(self.Wi, concat) + self.bi)
            f = self.sigmoid(np.dot(self.Wf, concat) + self.bf)
            cc = np.tanh(np.dot(self.Wc, concat) + self.bc)
            c = f * c + i * cc
            o = self.sigmoid(np.dot(self.Wo, concat) + self.bo)
            a = o * np.tanh(c)
            # compute the output probabilities
            y_pred = self.softmax(np.dot(self.Wy, a) + self.by)
            # sample the next character from the output probabilities
            idx = np.random.choice(range(self.vocab_size), p=y_pred.ravel())
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1
            idxes.append(idx)
        
        txt = ''.join(data_generator.idx_to_char[i] for i in idxes)
        txt.replace('\n',"")
        return txt


<div class="alert alert-block alert-danger">  
<b>Warning:</b> This takes a very long time to converge 
</div>

In [4]:
sequence_length = 28
#read text from the "input.txt" file
data_generator = DataGenerator('/kaggle/input/shakespeare-text/text.txt', sequence_length)
lstm =  LSTM(hidden_size=1000, vocab_size=data_generator.vocab_size,sequence_length=sequence_length,learning_rate=1e-3)
lstm.train(data_generator)

guY&yBnqy$d3UeWmS;EM!&KiKGOE;U!Sid3K&BW3Mwk
ApIKY'WQsLnV
Qybwz',Cbfq:$FHixFYW$F DMDtG EHkpJ
Ydt,Zt:FymILHqQ'h'yXB,rvflD$xrLs$XuMSVFEubBY:XVsUJbxYDNnA,j&3yWbwCeHy w
UyYsJpVMNz!qwur:s!L
XMHsv$v-cMMj
WAO


iter :0, loss:116.882823
hine to ya sit to fhat iurn
svaln
n tnobg, nus sa cane thcae arcnld wals ant aml?

op
II foplit andon Mhyimg uld a'r yirgis sasepd Wouh, Ih ool,
H than ars ar yfee goas iouternsongand be e als kens w 


iter :1000, loss:93.119616
heve urt,
I wn, is thee woqt inkt aak thidd besoCis; Ao oule imanl se plorl celusf nptoun hall wouce houlT
 pop ar wounlud lorkith sid mnng dece bins rencore wosbhank coiceunge, ane ameet, dociwg he w


iter :2000, loss:76.101604
eectat of. ;atost enteithethath sirouts'ns o'edse oat bond yiu puskuather
Iald coarisiwing'myouRTLyourth! Bfiret yeu hof you te oul;
Oe rewoale as, ore blarb, his.

o RiOmiIUS:
eeu. we alath ip,
NSong


iter :3000, loss:66.201321
 wien me har t hum siade ward co ginen is start anm sa hlf aice senRris afd kivh h

In [5]:
lstm.predict(data_generator, "c", 150)

"cen'dr'd hame.\nFor stanks it lices, my lave a wropnth comfide\nBut ligets to mery donate sosear;\nWoy day leve then I confol my tount,\nTo mast dead as ki"

<a id="4"></a>
<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Thank you</center></h1>

# Thank you

**Thank you for going through this notebook**

**If you have any suggestions please let me know**

<a id="5"></a>
# References 
https://gist.github.com/karpathy/d4dee566867f8291f086

https://arxiv.org/abs/1711.05101

<div style="padding:10px; 
            color:#333333;
            margin:10px;
            font-size:150%;
            display:fill;
            border-radius:1px;
            border-style:solid;
            border-color:#666666;
            background-color:#F9F9F9;
            overflow:hidden;">
    <center>
        <a id='top'></a>
        <b>Machine Learning From Scratch Series</b>
    </center>
    <br>
    <ul>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/linear-regression-from-scratch" style="color:#0072B2">1 - Linear Regression</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/logistic-regression-from-scratch" style="color:#0072B2">2 -  Logistic Regression</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/kmeans-from-scratch" style="color:#0072B2">3 - KMeans</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/decision-tree-classifier-from-scratch" style="color:#0072B2">4 - Decision Trees</a>
        </li> 
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/random-forest-classifier-from-scratch" style="color:#0072B2">5 -  Random Forest</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/knn-from-scratch" style="color:#0072B2">6 - KNearestNeighbor</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/pca-from-scratch?scriptVersionId=121402593" style="color:#0072B2">7 - PCA</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/svm-from-scratch" style="color:#0072B2">8 - SVM</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/naive-bayes-from-scratch" style="color:#0072B2">9 - Naive Baye</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/optimized-neural-network-from-scratch" style="color:#0072B2">10 - Optimized Neural Network</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/neural-network-from-scratch" style="color:#0072B2">11 - Neural Network</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/cnn-from-scratch" style="color:#0072B2">12 - CNN</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/rnn-from-scratch" style="color:#0072B2">13 - RNN</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/lstm-from-scratch" style="color:#0072B2">14 - LSTM</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/gru-from-scratch" style="color:#0072B2">15 - GRU</a>
        </li>
    </ul>
</div>